In [2]:
import sys
import ff

# sys.path.insert(0, "/Users/raymondiacobacci/Downloads/rcwaControl/")
# import rcwa  # type: ignore

import matplotlib.pyplot as plt
import numpy as np

import pandas as pd

h = 6.626070e-34  # Js Planck's constant
c = 2.997925e8  # m/s speed of light
k_B = 1.380649e-23  # J/K Boltzmann constant
q = 1.602176e-19  # C elementary charge
l_i = np.arange(350, 3001, 1)  # nm wavelength range
T_e = 2073.15  # K emitter temperature

blackbody_radiation = (
    lambda lambda_i, T: (2 * h * c**2)
    / ((np.exp((h * c) / (k_B * T * lambda_i * 1e-9)) - 1) * lambda_i**5)
    * 1e32
)

B_i = blackbody_radiation(l_i, T_e)  # 2073.15K blackbody

with open("homogeneous-film-emissivity.txt", "r") as f:
    p = f.readlines()
efficiency_array = ff.parse_data(p)

blackbody_transmission_array = [i * B_i for i in efficiency_array]

n_all = pd.read_excel("/Users/raymondiacobacci/LeiteLab-6_8/n-allHTMats-2.xlsx")
k_all = pd.read_excel("/Users/raymondiacobacci/LeiteLab-6_8/k-allHTMats.xlsx")

nb_B = (
    lambda lambda_i, T: (2 * c)
    / ((np.exp((h * c) / (k_B * T * lambda_i * 1e-9)) - 1) * lambda_i**4)
    * 1e23
)
nb_B_e = nb_B(l_i, T_e)
T_PV = 300
nb_B_PV = nb_B(l_i, T_PV)
nk_AlN = n_all["AlN"] + 1j * k_all["AlN"]
nk_W = n_all["W"] + 1j * k_all["W"]

# Testing of Kim's TORCWA

In [53]:
'''
This tests the performance of the two situations
'''
# from torcwa import rcwa

emissivity_at_wavelength_declan = np.load("control_density.npy")

def plot_density(array):
    print(emissivity_at_wavelength_declan)
    print(array * B_i)
    print(emissivity_at_wavelength_declan - array * B_i)
    
    plt.plot(l_i, array * B_i, label="+ Rand Grating")
    plt.plot(l_i, emissivity_at_wavelength_declan, label="Control (Declan)")
    plt.plot(l_i, B_i, label="Blackbody radiation")
    plt.plot(l_i, emissivity_at_wavelength_declan - array * B_i, label="Difference in values")
    plt.axvline(
        x=np.argmax(B_i) + l_i[0], color="r", linestyle="--", label="Peak intensity"
    )
    plt.legend(loc="upper right")
    plt.ylabel("Power Density (W/cm^2/nm)")
    plt.xlabel("Wavelength (nm)")
    plt.title("Spectral Density")
    plt.show()

er = np.ones(shape = (100,))
# er[50:60] = 0
ur = np.ones(shape=er.shape)
grating_depth = .473
grating_thickness = 1
latticeVectors = [grating_thickness, 1]

emissivity_at_wavelength = []

air_layer = rcwa.Layer(n = 1, thickness = 9999)
for wavelength_index in range(2651): # We have 2651 points
    # wavelength_index = 1150
    AlN_grating = rcwa.Crystal(latticeVectors, er=(nk_AlN[wavelength_index]**2-1) * er + 1, ur=ur)
    AlN_layer = rcwa.Layer(crystal = AlN_grating, thickness = grating_depth)
    W_layer = rcwa.Layer(n=nk_W[wavelength_index], thickness=9998)
    stack = rcwa.LayerStack(
        AlN_layer, incident_layer=air_layer, transmission_layer=W_layer
    )
    source = rcwa.Source(wavelength=l_i[wavelength_index] / 1000.0)
    solver = rcwa.Solver(stack, source, 3)
    results = solver.solve()
    emissivity_at_wavelength.append(1 - results['RTot'])
    # break

print(emissivity_at_wavelength)
# plot_density(emissivity_at_wavelength)

NameError: name 'rcwa' is not defined

# Testing of Raymond's rcwa-ctrl

In [ ]:
import sys
import ff

# sys.path.insert(0, "/Users/raymondiacobacci/Downloads/rcwaControl/")
# import rcwa  # type: ignore

import matplotlib.pyplot as plt
import numpy as np

import pandas as pd

h = 6.626070e-34  # Js Planck's constant
c = 2.997925e8  # m/s speed of light
k_B = 1.380649e-23  # J/K Boltzmann constant
q = 1.602176e-19  # C elementary charge
l_i = np.arange(350, 3001, 1)  # nm wavelength range
T_e = 2073.15  # K emitter temperature

blackbody_radiation = (
    lambda lambda_i, T: (2 * h * c**2)
    / ((np.exp((h * c) / (k_B * T * lambda_i * 1e-9)) - 1) * lambda_i**5)
    * 1e32
)

B_i = blackbody_radiation(l_i, T_e)  # 2073.15K blackbody

with open("homogeneous-film-emissivity.txt", "r") as f:
    p = f.readlines()
efficiency_array = ff.parse_data(p)

blackbody_transmission_array = [i * B_i for i in efficiency_array]

n_all = pd.read_excel("/Users/raymondiacobacci/LeiteLab-6_8/n-allHTMats-2.xlsx")
k_all = pd.read_excel("/Users/raymondiacobacci/LeiteLab-6_8/k-allHTMats.xlsx")

nb_B = (
    lambda lambda_i, T: (2 * c)
    / ((np.exp((h * c) / (k_B * T * lambda_i * 1e-9)) - 1) * lambda_i**4)
    * 1e23
)
nb_B_e = nb_B(l_i, T_e)
T_PV = 300
nb_B_PV = nb_B(l_i, T_PV)
nk_AlN = n_all["AlN"] + 1j * k_all["AlN"]
nk_W = n_all["W"] + 1j * k_all["W"]

In [355]:
%load_ext autoreload
%autoreload 2

import numpy as np
import solver_v2 as rc
import pandas as pd

h = 6.626070e-34  # Js Planck's constant
c = 2.997925e8  # m/s speed of light
k_B = 1.380649e-23  # J/K Boltzmann constant
q = 1.602176e-19  # C elementary charge
l_i = np.arange(350, 3001, 1)  # nm wavelength range
T_e = 2073.15  # K emitter temperature

n_all = pd.read_excel("/Users/raymondiacobacci/LeiteLab-6_8/n-allHTMats-2.xlsx")
k_all = pd.read_excel("/Users/raymondiacobacci/LeiteLab-6_8/k-allHTMats.xlsx")

nb_B = (
    lambda lambda_i, T: (2 * c)
    / ((np.exp((h * c) / (k_B * T * lambda_i * 1e-9)) - 1) * lambda_i**4)
    * 1e23
)
nb_B_e = nb_B(l_i, T_e)
T_PV = 300
nb_B_PV = nb_B(l_i, T_PV)
nk_AlN = n_all["AlN"] + 1j * k_all["AlN"]
nk_W = n_all["W"] + 1j * k_all["W"]

er = np.array([x%2 for x in range(100)])
er = np.ones(shape = er.shape)
ur = np.ones(shape = er.shape)

test_wavelength_index = 650

n_harmonics = 1

air_layer = rc.Layer_(permittivity = er, permeability = ur, thickness = 9, n_harmonics = n_harmonics)
aln_grating = rc.Layer_(
    permittivity=1*er,
    # permittivity=er*7,
    permeability=ur,
    thickness=.473,
    n_harmonics=n_harmonics,
)
w_layer = rc.Layer_(permittivity = nk_W[test_wavelength_index] ** 2 * er, permeability = ur, thickness = 9, n_harmonics = n_harmonics)

w_layer = rc.Layer_(permittivity = 1 ** 2 * er, permeability = ur, thickness = 9, n_harmonics = n_harmonics)

my_solver = rc.Solver_(layer_stack = [air_layer, aln_grating, w_layer, air_layer], grating_period = 1, wavelength = 1, n_harmonics = n_harmonics)
print(my_solver.soln())

# print(f'Emissivity: {my_solver.global_scattering_matrix[(2*n_harmonics + 1)*2:, :(2*n_harmonics + 1)*2]}')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Omega squared:
[[77.956835+0.j  0.      +0.j  0.      +0.j -0.      +0.j  0.      +0.j  0.      +0.j]
 [ 0.      +0.j -1.      +0.j  0.      +0.j  0.      +0.j  0.      +0.j  0.      +0.j]
 [ 0.      +0.j  0.      +0.j 77.956835+0.j  0.      +0.j  0.      +0.j -0.      +0.j]
 [ 0.      +0.j  0.      +0.j  0.      +0.j 77.956835+0.j  0.      +0.j  0.      +0.j]
 [ 0.      +0.j  0.      +0.j  0.      +0.j  0.      +0.j -1.      +0.j  0.      +0.j]
 [ 0.      +0.j  0.      +0.j  0.      +0.j  0.      +0.j  0.      +0.j 77.956835+0.j]]
Vacuum lambda (I have no idea if this is right or not at this point, I'm just copying the work from the middle layer's code)
[77.956835+0.j 77.956835-0.j 77.956835+0.j 77.956835-0.j -1.      +0.j -1.      +0.j]
Vac lambda w before reordering
[77.956835+0.j 77.956835-0.j 77.956835+0.j 77.956835-0.j -1.      +0.j -1.      +0.j]
Vacuum transfer matrix:
                    0 

In [297]:
7.795684e+01*0.707107+ -5.844232e-14*(0.687761-0.164269j)

(55.12382726187996+9.60026146408e-15j)

In [298]:
7.795684e+01*(0.414224-0.126281j)+ -5.844232e-14*(0.901372+0.000000j)

(32.29159409215995-9.84446771204j)

In [274]:
np.abs(0.9429905358928607+0.33281954452297624j)

0.999999999999993

In [190]:
0.85355339+0.02512627

0.87867966

In [191]:
0.17157288**2

0.029437253151494404

In [65]:
0.94299054**2+0.33281954j**2

(0.77846231232368+0j)

In [58]:
x=[[78,0,0,0,0,0],[0,-1,0,0,0,0],[0,0,78,0.1,0,0],[0,0,0,78,0,0],[0,0,0,0,-1,0],[0,0,0,0,0,78]]
print(x)
print(np.linalg.eig(x)[1])
print('Eigen computed')
print(x@np.linalg.eig(x)[1][:,3])
print('Value computed')
print(np.linalg.eig(x)[0][3]*np.linalg.eig(x)[1][:,3])
x1=[[ 7.79568352e+01+0.j , 0.00000000e+00+0.j , 0.00000000e+00+0.j,
  -5.84423150e-14+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j],
 [ 0.00000000e+00+0.j, -1.00000000e+00+0.j,  0.00000000e+00+0.j,
   0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j],
 [ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  7.79568352e+01+0.j,
   0.00000000e+00+0.j,  0.00000000e+00+0.j, -5.84423150e-14+0.j],
 [ 5.84423150e-14+0.j , 0.00000000e+00+0.j  ,0.00000000e+00+0.j
,   7.79568352e+01+0.j,  0.00000000e+00+0.j , 0.00000000e+00+0.j],
 [ 0.00000000e+00+0.j  ,0.00000000e+00+0.j  ,0.00000000e+00+0.j
 ,  0.00000000e+00+0.j, -1.00000000e+00+0.j,  0.00000000e+00+0.j],
 [ 0.00000000e+00+0.j ,0.00000000e+00+0.j ,5.84423150e-14+0.j
,0.00000000e+00+0.j ,0.00000000e+00+0.j ,7.79568352e+01+0.j]]
print(x1)
print(np.linalg.eig(x1)[1])
x2=[[ 7.79568352e+01+0.j , 0.00000000e+00+0.j , 0.00000000e+00+0.j,
  0+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j],
 [ 0.00000000e+00+0.j, -1.00000000e+00+0.j,  0.00000000e+00+0.j,
   0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j],
 [ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  7.79568352e+01+0.j,
   0.00000000e+00+0.j,  0.00000000e+00+0.j, 0+0.j],
 [ 0+0.j , 0.00000000e+00+0.j  ,0.00000000e+00+0.j
,   7.79568352e+01+0.j,  0.00000000e+00+0.j , 0.00000000e+00+0.j],
 [ 0.00000000e+00+0.j  ,0.00000000e+00+0.j  ,0.00000000e+00+0.j
 ,  0.00000000e+00+0.j, -1.00000000e+00+0.j,  0.00000000e+00+0.j],
 [ 0.00000000e+00+0.j ,0.00000000e+00+0.j ,0+0.j
,0.00000000e+00+0.j ,0.00000000e+00+0.j ,7.79568352e+01+0.j]]
print(x2)
print(np.linalg.eig(x2)[1])
print(np.sum(np.array(x1) - np.array(x2)))

[[78, 0, 0, 0, 0, 0], [0, -1, 0, 0, 0, 0], [0, 0, 78, 0.1, 0, 0], [0, 0, 0, 78, 0, 0], [0, 0, 0, 0, -1, 0], [0, 0, 0, 0, 0, 78]]
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00 -1.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.73194792e-13
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.00000000e+00]]
Eigen computed
[ 0.00000000e+00  0.00000000e+00 -7.80000000e+01  1.35091938e-11
  0.00000000e+00  0.00000000e+00]
Value computed
[ 0.00000000e+00  0.00000000e+00 -7.80000000e+01  1.35091938e-11
  0.00000000e+00  0.00000000e+00]
[[(77.9568352+0j), 0j, 0j, (-5.84423

In [62]:
p=[[ 39.4784176+0.j,   0.       +0.j,   0.       +0.j, -38.4784176+0.j,
    0.       +0.j,   0.       +0.j],
 [  0.       +0.j,   0.       +0.j,   0.       +0.j,   0.       +0.j,
    1.       +0.j,   0.       +0.j],
 [  0.       +0.j,   0.       +0.j,  39.4784176+0.j,   0.       +0.j,
    0.       +0.j, -38.4784176+0.j],
 [ 38.4784176+0.j,   0.       +0.j,   0.       +0.j, -39.4784176+0.j,
    0.       +0.j,   0.       +0.j],
 [  0.       +0.j,  -1.       +0.j,   0.       +0.j,   0.       +0.j,
    0.       +0.j,   0.       +0.j],
 [  0.       +0.j,   0.       +0.j,  38.4784176+0.j,   0.       +0.j,
    0.       +0.j, -39.4784176+0.j]]
q=p=[[ 39.4784176+0.j,   0.       +0.j,   0.       +0.j, -38.4784176+0.j,
    0.       +0.j,   0.       +0.j],
 [  0.       +0.j,   0.       +0.j,   0.       +0.j,   0.       +0.j,
    1.       +0.j,   0.       +0.j],
 [  0.       +0.j,   0.       +0.j,  39.4784176+0.j,   0.       +0.j,
    0.       +0.j, -38.4784176+0.j],
 [ 38.4784176+0.j,   0.       +0.j,   0.       +0.j, -39.4784176+0.j,
    0.       +0.j,   0.       +0.j],
 [  0.       +0.j,  -1.       +0.j,   0.       +0.j,   0.       +0.j,
    0.       +0.j,   0.       +0.j],
 [  0.       +0.j,   0.       +0.j,  38.4784176+0.j,   0.       +0.j,
    0.       +0.j, -39.4784176+0.j]]
p,q=np.array(p),np.array(q)
print(p@q)
print(np.matmul(p,q))
print(manual_matmul(p,q))

[[ 7.79568352e+01+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   7.64441833e-14+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j -1.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j  7.79568352e+01+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  7.64441833e-14+0.j]
 [-7.64441833e-14+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   7.79568352e+01+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j -1.00000000e+00+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j -7.64441833e-14+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  7.79568352e+01+0.j]]
[[ 7.79568352e+01+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   7.64441833e-14+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j -1.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j]

In [61]:
def manual_matmul(A, B):
    """
    Manually performs matrix multiplication between two NumPy arrays A and B.

    Parameters:
    - A: NumPy array of shape (m, n)
    - B: NumPy array of shape (n, p)

    Returns:
    - result: NumPy array of shape (m, p) resulting from A x B
    """

    # Get the dimensions of the input matrices
    a_rows, a_cols = A.shape
    b_rows, b_cols = B.shape

    # Check if the matrices can be multiplied
    if a_cols != b_rows:
        raise ValueError("Incompatible dimensions for matrix multiplication.")

    # Initialize the result matrix with zeros
    result = np.zeros((a_rows, b_cols), dtype=complex)

    # Perform the matrix multiplication manually
    for i in range(a_rows):
        for j in range(b_cols):
            for k in range(a_cols):  # or range(b_rows)
                result[i, j] += A[i, k] * B[k, j]
    return result
